###  DOCUMENTATION:  
    https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html

### Initiate Spark 

control a Spark Application through a driver process called SparkSession

1) on console: ```spark```

2) on jupyternotebook: ```jupyter notebook``` then in a cell run


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

### Spark UI 

```http://localhost:4040/jobs/```

#todo to 5 

## Ch5: Basic Structured Operations

### Dataframes Schemas

1) schema-on-read (autodetect)

2) defined explicitly

In [2]:
# to load and check the schema  without schema(myManulaSchema)
spark.read.format('json').load('./data/flight-data/json/2015-summary.json').schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

In [4]:
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
  StructField("DEST_COUNTRY_NAME", StringType(), True),
  StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
  StructField("count", LongType(), False, metadata={"hello":"world"})
])
flights_df = spark.read.format("json").schema(myManualSchema)\
  .load("./data/flight-data/json/2015-summary.json")

flights_df .schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

A schema is a ```StructType``` build by ```StructField``` made of:

    1) nameColumn
    
    2) typeColumn
    
    3) Nullable
    
    4) metadata (optional)

To check the schema 

In [5]:
flights_df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



### Columns

The book combines the Scala and PySpark API's.

In Scala / Java API, ```df.col("column_name")```,```df.col('column_name)```,```df("column_name")``` or  ```df.apply("column_name")``` return the Column.

Whereas in pyspark use the below to get the column from DF.

```df.colName```
```df["colName"]```

<b>HOWEVER </b>, if using  ```select``` it is also possible to use ```col("column_name")```

In [6]:
from pyspark.sql.functions import col, column

# df("someColumnName")
flights_df["DEST_COUNTRY_NAME"]
flights_df.DEST_COUNTRY_NAME


Column<'DEST_COUNTRY_NAME'>

In [7]:
flights_df.columns #column property to access columns 

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

## Expressions 

an expression parses transformations and column references from a string 

In [8]:
from pyspark.sql.functions import expr

df = spark.range(500).toDF("number")
df.select(df["number"] + 10).take(3)
 
df.select(expr("(((number + 5) * 200) - 6) < 5")).take(3)

[Row(((((number + 5) * 200) - 6) < 5)=False),
 Row(((((number + 5) * 200) - 6) < 5)=False),
 Row(((((number + 5) * 200) - 6) < 5)=False)]

### Rows
Each row is a single record, represented as an object of type ```Row```. To manipulate an object of type ```Row``` use a column expression (previous paragraph). Internally represent arrays of bytes.

In [9]:
df.first() # an example to check a type Row is printing

Row(number=0)

#### Create Rows
1) manually instanciatin an object ```Row``` (values in the same order and type as the schema of the df to which you have to append them


In [10]:
from pyspark.sql import Row
myRow = Row("Hello", None, 1, False)

In [11]:
myRow[0] # to access the value

'Hello'

### DataFrame

#### Creating df
1) from a file / raw data sources ```spark.read.format('format').source('path/to/data')```
2) from a set of rows 

In [12]:
from pyspark import Row
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
  StructField("Welcome", StringType(), True),
  StructField("None", StringType(), True),
  StructField("number", LongType(), False, metadata={"hello":"world"})
])

myRow = Row("Hello", None, 1)
myDf = spark.createDataFrame([myRow], myManualSchema)
myDf.show()

+-------+----+------+
|Welcome|None|number|
+-------+----+------+
|  Hello|null|     1|
+-------+----+------+



#### Transforming a Df
To transform a Df we can only manipulate columns (rows singularly are not accessible) and we can use 
1) ```select``` method

2) ```selectExpr``` method

3) ```import pyspark.sql.functions``` package

#### Transforming using SELECT

In [13]:
flights_df.select("DEST_COUNTRY_NAME").show(2) # singular selection

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [14]:
flights_df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2) #multiple selection


+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



In [15]:
from pyspark.sql.functions import expr, column, col 
flights_df.select(expr("DEST_COUNTRY_NAME"),
                 col("ORIGIN_COUNTRY_NAME"),
#                  column("count")               # column is not working 
                 ).show(5)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
|    United States|            Ireland|
|            Egypt|      United States|
|    United States|              India|
+-----------------+-------------------+
only showing top 5 rows



<b> NOT TRUE common mistake </b>: use a mix of column Objects and strings, does not give an error

In [16]:
from pyspark.sql.functions import expr, column, col 
flights_df.select(
                 col("ORIGIN_COUNTRY_NAME"),"DEST_COUNTRY_NAME"
                 ).show(5)

+-------------------+-----------------+
|ORIGIN_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-------------------+-----------------+
|            Romania|    United States|
|            Croatia|    United States|
|            Ireland|    United States|
|      United States|            Egypt|
|              India|    United States|
+-------------------+-----------------+
only showing top 5 rows



#### Rename columns 

In [17]:
flights_df.select(expr("ORIGIN_COUNTRY_NAME AS origin")).show(2)
flights_df.select(expr("ORIGIN_COUNTRY_NAME").alias("origin2")).show(2) #### NB the alias is INSIDE select

+-------+
| origin|
+-------+
|Romania|
|Croatia|
+-------+
only showing top 2 rows

+-------+
|origin2|
+-------+
|Romania|
|Croatia|
+-------+
only showing top 2 rows



#### Transforming using .selectExpr()
Because ```select``` and ```expr``` is a commone pattern --> short hand ```selectExpr```

In [18]:
flights_df.selectExpr("ORIGIN_COUNTRY_NAME AS origin").show(2)

+-------+
| origin|
+-------+
|Romania|
|Croatia|
+-------+
only showing top 2 rows



In [19]:
flights_df.selectExpr("*","ORIGIN_COUNTRY_NAME AS origin", "ORIGIN_COUNTRY_NAME = DEST_COUNTRY_NAME as withInCountry").show(2)

+-----------------+-------------------+-----+-------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count| origin|withInCountry|
+-----------------+-------------------+-----+-------+-------------+
|    United States|            Romania|   15|Romania|        false|
|    United States|            Croatia|    1|Croatia|        false|
+-----------------+-------------------+-----+-------+-------------+
only showing top 2 rows



###  Literals
```lit``` is used to pass explicit values into Spark that are just a value. Need to be imported 

In [20]:
from pyspark.sql.functions import lit 
flights_df.select(expr("*"), lit(True)).show(2) # lit is OUTSIDE expr()
flights_df.select(expr("*"), lit(True).alias("True value?")).show(2) # lit is OUTSIDE expr()

#NB a difference with SCALA is that .alias() in Python is like .as() in Scala

+-----------------+-------------------+-----+----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|true|
+-----------------+-------------------+-----+----+
|    United States|            Romania|   15|true|
|    United States|            Croatia|    1|true|
+-----------------+-------------------+-----+----+
only showing top 2 rows

+-----------------+-------------------+-----+-----------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|True value?|
+-----------------+-------------------+-----+-----------+
|    United States|            Romania|   15|       true|
|    United States|            Croatia|    1|       true|
+-----------------+-------------------+-----+-----------+
only showing top 2 rows



### Adding Columns with ```withColumns('column_name', value)``` 

In [21]:
flights_df.withColumn('numberOne', lit(1)).show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [22]:
# withColumn(column_name, expression)
flights_df.withColumn('withInCountry', expr("DEST_COUNTRY_NAME=ORIGIN_COUNTRY_NAME")).show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withInCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



### Rename a column ```withColumnRenamed('old_name', 'new_name')```

In [23]:
new_flights_df= flights_df.withColumnRenamed("DEST_COUNTRY_NAME", 'destination')
new_flights_df.show(2)

+-------------+-------------------+-----+
|  destination|ORIGIN_COUNTRY_NAME|count|
+-------------+-------------------+-----+
|United States|            Romania|   15|
|United States|            Croatia|    1|
+-------------+-------------------+-----+
only showing top 2 rows



### Remove columns


In [24]:
new_flights_df=new_flights_df.drop('destination')
new_flights_df.show(2)

+-------------------+-----+
|ORIGIN_COUNTRY_NAME|count|
+-------------------+-----+
|            Romania|   15|
|            Croatia|    1|
+-------------------+-----+
only showing top 2 rows



### Changing Column Type ```cast('type')```

In [25]:
new_flights_df.withColumn("StringNumber", col('count').cast('string')).show(2)
new_flights_df.withColumn("StringNumber", col('count').cast('string')).printSchema()

+-------------------+-----+------------+
|ORIGIN_COUNTRY_NAME|count|StringNumber|
+-------------------+-----+------------+
|            Romania|   15|          15|
|            Croatia|    1|           1|
+-------------------+-----+------------+
only showing top 2 rows

root
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)
 |-- StringNumber: string (nullable = true)



### Filtering ```where()``` or ```filter()```

There are two methods to perform this operation: you can use where or filter
and they both will perform the same operation and accept the same argument types when used
with DataFrames.

In [26]:
new_flights_df.filter(col("count") < 2 ).show(2)

+-------------------+-----+
|ORIGIN_COUNTRY_NAME|count|
+-------------------+-----+
|            Croatia|    1|
|          Singapore|    1|
+-------------------+-----+
only showing top 2 rows



In [27]:
new_flights_df.where(col("count") < 2 ).show(2)
new_flights_df.where(expr("count")< 2 ).show(2)
# new_flights_df.where("count"< 2 ).show(2) # NOT working is comapring strign and number 

+-------------------+-----+
|ORIGIN_COUNTRY_NAME|count|
+-------------------+-----+
|            Croatia|    1|
|          Singapore|    1|
+-------------------+-----+
only showing top 2 rows

+-------------------+-----+
|ORIGIN_COUNTRY_NAME|count|
+-------------------+-----+
|            Croatia|    1|
|          Singapore|    1|
+-------------------+-----+
only showing top 2 rows



<b> NOT use multiple filters into the same expression.</b> Although this is
possible, it is not always useful, because Spark automatically performs all filtering operations at
the same time regardless of the filter ordering. This means that if you want to specify multiple
AND filters, <b> just chain them sequentially 

In [36]:
new_flights_df.where(col("count") >2).where(col("ORIGIN_COUNTRY_NAME") == "Romania" ).show(7)


+-------------------+-----+
|ORIGIN_COUNTRY_NAME|count|
+-------------------+-----+
|            Romania|   15|
+-------------------+-----+



### Getting Unique Rows: df.distinct()

In [39]:
new_flights_df.distinct().count()

220

In [41]:
new_flights_df.distinct().orderBy("count").show(5)

+-------------------+-----+
|ORIGIN_COUNTRY_NAME|count|
+-------------------+-----+
|          Lithuania|    1|
|          Singapore|    1|
|          Gibraltar|    1|
|           Bulgaria|    1|
|            Namibia|    1|
+-------------------+-----+
only showing top 5 rows



### Filtering by Rows

In [88]:
new_flights_df.take(10)


[Row(ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(ORIGIN_COUNTRY_NAME='India', count=62),
 Row(ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(ORIGIN_COUNTRY_NAME='United States', count=1)]

.take() results in an Array of Rows. This is an action and performs collecting the data (like collect does).



In [89]:
flights_df.limit(10)


DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

In [90]:
flights_df.limit(10).show()


+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+



limit() results in a new Dataframe. This is a transformation and does not perform collecting the data.


### Random Sample: df.sample()
sample some random records from your DataFrame. sample(withReplacement=None, fraction=None, seed=None)
This is not guaranteed to provide exactly the fraction specified of the total count of the given DataFrame.

In [51]:
# in Python
seed = 5   
withReplacement = False #Sample with replacement or not (default False).
fraction = 0.5  # hFraction of rows to generate, range [0.0, 1.0].
new_flights_df.sample(withReplacement, fraction, seed).count()

138

In [52]:
new_flights_df.count()

256

In [54]:
new_flights_df.count()*fraction

128.0

### Random Splits .randomSplit([0.25, 0.75], seed)
Random splits can be helpful when you need to break up your DataFrame into a random “splits” of the original DataFrame. 

Parameters:

weights: list --> list of doubles as weights with which to split the DataFrame. Weights will be normalized if they don’t sum up to 1.0.

seed: int, optional  --> 
The seed for sampling.

In [61]:
dataFrames = new_flights_df.randomSplit([0.25, 0.75], seed)
dataFrames[0].show(3)
dataFrames[0].count()

+-------------------+-----+
|ORIGIN_COUNTRY_NAME|count|
+-------------------+-----+
|             Angola|   13|
|           Anguilla|   38|
|          Australia|  258|
+-------------------+-----+
only showing top 3 rows



71

In [62]:
dataFrames[1].show(3)
dataFrames[1].count()

+-------------------+-----+
|ORIGIN_COUNTRY_NAME|count|
+-------------------+-----+
|Antigua and Barbuda|  117|
|          Argentina|  141|
|              Aruba|  342|
+-------------------+-----+
only showing top 3 rows



185

### Concatenating and Appending Rows (Union)

In [71]:
# in Python
from pyspark.sql import Row
schema = flights_df.schema
newRows = [
Row("New Country", "Other Country", 5), Row("New Country 2", "Other Country 3", 1)
]
#Just drop the L; all integers in Python 3 are long. What was long in Python 2 is now the standard int type in Python 3.
parallelizedRows = spark.sparkContext.parallelize(newRows)
newDF = spark.createDataFrame(parallelizedRows, schema)
newDF.show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|      New Country|      Other Country|    5|
|    New Country 2|    Other Country 3|    1|
+-----------------+-------------------+-----+



In [91]:
reduced = flights_df.limit(2)
reduced.union(newDF).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|      New Country|      Other Country|    5|
|    New Country 2|    Other Country 3|    1|
+-----------------+-------------------+-----+



### Sorting Rows

sort
and orderBy that work the exact same way. They accept both column expressions and strings as
well as multiple columns. The default is to sort in ascending order:

In [97]:
flights_df.orderBy(["ORIGIN_COUNTRY_NAME","DEST_COUNTRY_NAME"], ascending=False).show()

+--------------------+-------------------+------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+--------------------+-------------------+------+
|       United States|            Vietnam|     2|
|       United States|          Venezuela|   246|
|       United States|            Uruguay|    13|
|              Zambia|      United States|     1|
|           Venezuela|      United States|   290|
|             Uruguay|      United States|    43|
|       United States|      United States|370002|
|      United Kingdom|      United States|  2025|
|United Arab Emirates|      United States|   320|
|             Ukraine|      United States|    14|
|Turks and Caicos ...|      United States|   230|
|              Turkey|      United States|   138|
|             Tunisia|      United States|     3|
| Trinidad and Tobago|      United States|   211|
|         The Bahamas|      United States|   955|
|            Thailand|      United States|     3|
|              Taiwan|      United States|   266|


In [113]:
flights_df.sort(["ORIGIN_COUNTRY_NAME","DEST_COUNTRY_NAME"], ascending=False).show(3)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Vietnam|    2|
|    United States|          Venezuela|  246|
|    United States|            Uruguay|   13|
+-----------------+-------------------+-----+
only showing top 3 rows



### asc_nulls_first, desc_nulls_first, asc_nulls_last, or desc_nulls_last 

to specify where you would like your null values to appear in an ordered
DataFrame. Returns a sort expression based on ascending order of the column, and null values return before non-null values.

In [119]:
# check the syntax df.orderBy(df.column.asc_nulls_last())

flights_df.orderBy(flights_df.DEST_COUNTRY_NAME.asc_nulls_last()).show(3)



+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Algeria|      United States|    4|
|           Angola|      United States|   15|
|         Anguilla|      United States|   41|
+-----------------+-------------------+-----+
only showing top 3 rows



### sortWithinPartition vs orderBy vs sort 

The documentation of sortWithinPartition states it returns a new Dataset with each partition sorted by the given expressions

The easiest way to think of this function is to imagine a fourth column (the partition id) that is used as primary sorting criterion. The function spark_partition_id() prints the partition.

For example if you have just one large partition (something that you as a Spark user would never do!), sortWithinPartition works as a normal sort:



In [110]:
from pyspark.sql.functions import spark_partition_id
# This is non deterministic because it depends on data partitioning and task scheduling.
#https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.spark_partition_id.html


flights_df.repartition(1)\
        .sortWithinPartitions(["ORIGIN_COUNTRY_NAME","DEST_COUNTRY_NAME"],\
                                               ascending=False)\
        .withColumn("partition", spark_partition_id()).show()

+--------------------+-------------------+------+---------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|partition|
+--------------------+-------------------+------+---------+
|       United States|            Vietnam|     2|        0|
|       United States|          Venezuela|   246|        0|
|       United States|            Uruguay|    13|        0|
|              Zambia|      United States|     1|        0|
|           Venezuela|      United States|   290|        0|
|             Uruguay|      United States|    43|        0|
|       United States|      United States|370002|        0|
|      United Kingdom|      United States|  2025|        0|
|United Arab Emirates|      United States|   320|        0|
|             Ukraine|      United States|    14|        0|
|Turks and Caicos ...|      United States|   230|        0|
|              Turkey|      United States|   138|        0|
|             Tunisia|      United States|     3|        0|
| Trinidad and Tobago|      United State


If there are more partitions, the results are only sorted within each partition:


Why would one use sortWithPartition instead of sort? 

<b> sortWithPartition does not trigger a shuffle </b>, as the data is only moved within the executors. sort however will trigger a shuffle. Therefore sortWithPartition executes faster. If the data is partitioned by a meaningful column, sorting within each partition might be enough.

In [111]:
flights_df.repartition(90)\
        .sortWithinPartitions(["ORIGIN_COUNTRY_NAME","DEST_COUNTRY_NAME"],\
                                               ascending=False)\
        .withColumn("partition", spark_partition_id()).show()

+-------------------+-------------------+-----+---------+
|  DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|partition|
+-------------------+-------------------+-----+---------+
|   Marshall Islands|      United States|   42|        0|
|      United States|              Italy|  438|        0|
|      United States|           Anguilla|   38|        0|
|            Jamaica|      United States|  666|        1|
|            Hungary|      United States|    2|        1|
|      United States|              Qatar|  109|        1|
|         Luxembourg|      United States|  155|        2|
|              India|      United States|   61|        2|
|      United States|       Cook Islands|   13|        2|
|     United Kingdom|      United States| 2025|        3|
|           Kiribati|      United States|   26|        3|
|      United States|          Argentina|  141|        3|
|            Uruguay|      United States|   43|        4|
|         Guadeloupe|      United States|   56|        4|
|      French 

### Repartition

Repartition will incur a full shuffle of the data, regardless of whether one is necessary. This means that you should typically only repartition when the future number of partitions is greater than your current number of partitions.


If you know that you’re going to be filtering by a certain column often, it can be worth
repartitioning based on that column:

In [120]:
# in Python
flights_df.repartition(5, col("DEST_COUNTRY_NAME"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

### Coalesce
will not incur a full shuffle and will try to combine partitions. This operation will shuffle your data into five partitions based on the destination country name, and then coalesce them (without a full shuffle):

In [ ]:
# in Python
df.repartition(5, col("DEST_COUNTRY_NAME")).coalesce(2)

###

In [121]:
# in Python
collectDF = flights_df.limit(10)
collectDF.take(5) # take works with an Integer count
collectDF.show() # this prints it out nicely
collectDF.show(5, False)
collectDF.collect()
collectDF.toLocalIterator()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India         

<generator object _local_iterator_from_socket.<locals>.PyLocalIterable.__iter__ at 0x7f21ff565510>

In [124]:
collectDF.StorageLevel()

AttributeError: 'DataFrame' object has no attribute 'StorageLevel'

In [28]:
new_flights_df.limit(3).show()

+-------------------+-----+
|ORIGIN_COUNTRY_NAME|count|
+-------------------+-----+
|            Romania|   15|
|            Croatia|    1|
|            Ireland|  344|
+-------------------+-----+



In [63]:
#new_flights_df.where(col("count") >2 | col("ORIGIN_COUNTRY_NAME") == "Romania" ).show(7)
#use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

###

###

###

In [41]:
df.select(expr("(((column('number') + 5) * 200) - 6) < 5")).take(3)

AnalysisException: Undefined function: 'column'. This function is neither a registered temporary function nor a permanent function registered in the database 'default'.; line 1 pos 3